# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Backfill Features to the Feature Store</span>


## 🗒️ This notebook is divided in 3 sections:
1. Loading the data 
2. Connect to the Hopsworks feature store,
3. Create feature groups and insert them to the feature store.

![tutorial-flow](images/01_featuregroups.png)

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import pandas as pd

from features import *

## <span style='color:#ff5f27'> 💽 Loading Historical Data</span>


#### <span style='color:#ff5f27'> 👩🏻‍🔬 Air Quality Data

In [ ]:
df_air_quality = pd.read_csv('data/air_quality.csv')

df_air_quality.head()

In [ ]:
df_air_quality.date = df_air_quality.date.apply(timestamp_2_time)
df_air_quality.sort_values(by = ['city','date'],inplace = True,ignore_index = True)

df_air_quality.head()

#### <span style='color:#ff5f27'> 🌦 Weather Data

In [ ]:
df_weather = pd.read_csv('data/weather.csv')

df_weather.head()

In [ ]:
df_weather.date = df_weather.date.apply(timestamp_2_time)
df_weather.sort_values(by = ['city','date'],inplace = True,ignore_index = True)

df_weather.head()

## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

## <span style="color:#ff5f27;">🪄 Creating Feature Groups</span>

#### <span style='color:#ff5f27'> 👩🏻‍🔬 Air Quality Data

In [ ]:
air_quality_fg = fs.get_or_create_feature_group(
        name = 'air_quality_fg',
        description = 'Air Quality characteristics of each day',
        version = 1,
        primary_key = ['city','date'],
        online_enabled = True,
        event_time = ['date']
    )    

air_quality_fg.insert(df_air_quality)

#### <span style='color:#ff5f27'> 🌦 Weather Data

In [ ]:
weather_fg = fs.get_or_create_feature_group(
        name = 'weather_fg',
        description = 'Weather characteristics of each day',
        version = 1,
        primary_key = ['city','date'],
        online_enabled = True,
        event_time = ['date']
    )    

weather_fg.insert(df_weather)

In [5]:
weather_df = weather_fg.select_all().read().sort_values(by=["date"])

2022-09-20 10:38:58,819 INFO: USE `maksym00_featurestore`
2022-09-20 10:39:00,317 INFO: SELECT `fg0`.`city` `city`, `fg0`.`date` `date`, `fg0`.`tempmax` `tempmax`, `fg0`.`tempmin` `tempmin`, `fg0`.`temp` `temp`, `fg0`.`feelslikemax` `feelslikemax`, `fg0`.`feelslikemin` `feelslikemin`, `fg0`.`feelslike` `feelslike`, `fg0`.`dew` `dew`, `fg0`.`humidity` `humidity`, `fg0`.`precip` `precip`, `fg0`.`precipprob` `precipprob`, `fg0`.`precipcover` `precipcover`, `fg0`.`snow` `snow`, `fg0`.`snowdepth` `snowdepth`, `fg0`.`windgust` `windgust`, `fg0`.`windspeed` `windspeed`, `fg0`.`winddir` `winddir`, `fg0`.`pressure` `pressure`, `fg0`.`cloudcover` `cloudcover`, `fg0`.`visibility` `visibility`, `fg0`.`solarradiation` `solarradiation`, `fg0`.`solarenergy` `solarenergy`, `fg0`.`uvindex` `uvindex`, `fg0`.`conditions` `conditions`
FROM `maksym00_featurestore`.`weather_fg_1` `fg0`


In [6]:
weather_df

,city,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions
12,Sundsvall,1662411600000,13.0,3.0,8.6,13.0,0.1,7.4,5.5,81.8,...,31.0,14.3,192.9,1024.1,90.8,15.3,116.1,10.1,5.0,Overcast
5,Stockholm,1662411600000,15.9,7.8,12.0,15.9,7.1,11.8,7.0,73.6,...,25.2,13.0,70.7,1022.0,59.5,15.3,132.5,11.6,5.0,Partially cloudy
6,Malmo,1662411600000,20.6,12.4,15.9,20.6,12.4,15.9,9.3,66.6,...,44.6,23.7,97.9,1018.4,69.0,15.2,157.3,13.7,5.0,Partially cloudy
15,Kyiv,1662411600000,17.7,4.6,11.5,17.7,4.6,11.5,1.8,55.3,...,24.5,9.7,267.0,1022.3,34.8,24.1,227.5,19.6,7.0,Partially cloudy
2,Malmo,1662498000000,20.2,12.4,15.8,20.2,12.4,15.8,9.3,66.6,...,44.6,25.6,97.9,1018.4,67.2,14.6,168.5,14.6,6.0,Partially cloudy
3,Kyiv,1662498000000,17.7,4.6,11.5,17.7,4.6,11.5,1.8,55.3,...,24.5,9.7,267.0,1022.3,34.8,24.1,227.5,19.6,7.0,Partially cloudy
7,Sundsvall,1662498000000,13.0,3.0,8.6,13.0,0.1,7.4,5.6,81.9,...,31.0,14.3,192.9,1024.1,90.8,14.7,116.1,10.1,5.0,Overcast
14,Stockholm,1662498000000,15.9,7.8,12.0,15.9,7.1,11.8,7.0,73.5,...,25.2,13.0,69.9,1022.0,57.5,14.7,142.5,12.5,5.0,Partially cloudy
1,Malmo,1662584400000,20.2,14.5,16.9,20.2,14.5,16.9,10.0,65.1,...,60.5,28.4,94.1,1015.0,93.3,20.6,91.8,7.8,5.0,"Rain, Overcast"
4,Kyiv,1662584400000,21.1,7.9,14.3,21.1,7.9,14.3,3.3,50.9,...,24.8,9.7,132.9,1019.4,48.4,24.1,217.9,18.7,7.0,Partially cloudy
